# `sc_linac` Live

In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
from lcls_live.datamaps import get_datamaps
from lcls_live.archiver import lcls_archiver_restore

from lcls_live.tools import isotime

import matplotlib.pyplot as plt
import numpy as np

import os

In [3]:
# Pick a model and slice

MODEL = 'sc_hxr' # or sc_diag0 or sc_inj

BEGELE = 'BEGINNING'
#ENDELE = 'OTR0H04'
ENDELE = 'ENDCOL0'

# Bmad model

In [4]:
from pytao import Tao
import pandas as pd

In [5]:
tao = Tao(f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init  -slice  {BEGELE}:{ENDELE}')
tao.cmd('place floor bpm_orbit')
tao.cmd('place middle energy')
tao.cmd('sc floor -10 10')

def ele_info(ele):
    dat = tao.ele_head(ele)
    dat.update(tao.ele_gen_attribs(ele))
    return dat

def ele_table(match="*"):
    ix_ele = tao.lat_list(match, "ele.ix_ele", flags="-no_slaves")
    dat = list(map(ele_info, ix_ele))
    df = pd.DataFrame(dat, index=ix_ele)
    df.L.fillna(0, inplace=True)
    df['s_center'] = df['s'] - df['L']/2
    df['s_beginning'] = df['s'] - df['L']
    return  df

df = ele_table()

# Elements with device names
devices = df[df['alias'] != '']
#devices['name alias s'.split()]

## Datamaps, and all PVs needed

In [6]:
from copy import deepcopy
def filter_datamap(dm, bmad_names):
    bnames = dm.data['bmad_name'] 
    bmad_names = set(bmad_names)
    ix = bnames[[name in bmad_names for name in bnames]].index
    dm2 = deepcopy(dm)
    dm2.data = dm.data.loc[ix]
    return dm2

In [7]:
DM0 = get_datamaps(MODEL)
DM0.keys()

dict_keys(['bpms', 'cavities', 'correctors', 'tao_energy_measurements', 'quad', 'solenoid'])

In [8]:
good_names = set(df['name'])
bad_eles = [] # any bad eles

for ele in bad_eles:
    good_names.remove(ele)

DM = {}
for name, dm in DM0.items():
    if name == 'tao_energy_measurements':
        # don't filter
        DM[name] = dm
    else:
        DM[name] = filter_datamap(dm, good_names)

DM['bpms']

TabularDataMap(data=                      pvname         tao_datum    tao_factor bmad_name
0       BPMS:GUNB:314:XSCH1H        orbit.x[1]  1.000000e-03     BPM1B
1       BPMS:GUNB:925:XSCH1H        orbit.x[2]  1.000000e-03     BPM2B
2        BPMS:HTR:120:XSCH1H        orbit.x[3]  1.000000e-03   BPM0H01
3        BPMS:HTR:320:XSCH1H        orbit.x[4]  1.000000e-03   BPM0H04
4        BPMS:HTR:365:XSCH1H        orbit.x[5]  1.000000e-03   BPM0H05
..                       ...               ...           ...       ...
362  BPMS:COL0:640:TMITSCH1H  orbit.charge[21]  1.602177e-19   BPMC008
363  BPMS:COL0:720:TMITSCH1H  orbit.charge[22]  1.602177e-19   BPMC009
364  BPMS:COL0:800:TMITSCH1H  orbit.charge[23]  1.602177e-19   BPMC010
365  BPMS:COL0:880:TMITSCH1H  orbit.charge[24]  1.602177e-19   BPMC011
366  BPMS:COL0:940:TMITSCH1H  orbit.charge[25]  1.602177e-19   BPMC012

[75 rows x 4 columns], pvname='pvname', element='tao_datum', attribute='', factor='tao_factor', offset='', bmad_format='! No eq

In [9]:
# datamaps to exclude
DENYLIST = [
    #'bpms',
   # 'cavities',
    'correctors',
    #'quad',
]

In [10]:
# PVs needed
PVLIST =  []
for name, dm in DM.items():
    if name in DENYLIST:
        continue
    PVLIST.extend(dm.pvlist)
PVLIST = list(set(PVLIST))
len(PVLIST)
PVLIST[0:10]

['BPMS:COL0:240:YSCH1H',
 'ACCL:L0B:0120:AACTMEAN',
 'BPMS:COL0:880:XSCH1H',
 'SOLN:GUNB:100:BACT',
 'BPMS:COL0:260:XSCH1H',
 'BPMS:HTR:830:TMITSCH1H',
 'BPMS:GUNB:925:YSCH1H',
 'QUAD:COL0:480:BACT',
 'ACCL:L0B:0110:PACTMEAN',
 'BPMS:COL0:320:XSCH1H']

## EPICS

In [11]:
import epics
from epics import caget_many, caget
from time import sleep, time

def caget_dict(pvlist):
    return dict(zip(pvlist, caget_many(pvlist)))

In [12]:
# Test get
PVDATA = caget_dict(PVLIST)
BAD_DEVICES = set()
PVLIST_GOOD = []
for k, v in PVDATA.items():
    if v is None:
        print('Bad PV:', k)
        device = ':'.join((k.split(':')[:-1]))
        BAD_DEVICES.add(device)
    else:
        PVLIST_GOOD.append(k)
        
# Get bmad names
bdf = devices[['alias', 'name']].set_index('alias')
BAD_NAMES = list(bdf.loc[list(BAD_DEVICES)]['name'])
BAD_NAMES        
        
BAD_DEVICES, BAD_NAMES   

Bad PV: BPMS:COL0:260:XSCH1H
Bad PV: BPMS:COL0:720:XSCH1H
Bad PV: BPMS:COL0:480:TMITSCH1H
Bad PV: BPMS:COL0:720:YSCH1H
Bad PV: BPMS:HTR:980:TMITSCH1H
Bad PV: BPMS:COL0:800:TMITSCH1H
Bad PV: BPMS:COL0:480:XSCH1H
Bad PV: BPMS:COL0:800:YSCH1H
Bad PV: BPMS:HTR:980:YSCH1H
Bad PV: BPMS:COL0:260:TMITSCH1H
Bad PV: BPMS:COL0:480:YSCH1H
Bad PV: BPMS:COL0:720:TMITSCH1H
Bad PV: BPMS:COL0:800:XSCH1H
Bad PV: BPMS:HTR:980:XSCH1H
Bad PV: BPMS:COL0:260:YSCH1H


({'BPMS:COL0:260',
  'BPMS:COL0:480',
  'BPMS:COL0:720',
  'BPMS:COL0:800',
  'BPMS:HTR:980'},
 ['BPMC006', 'BPMC009', 'BPMHD04', 'BPMC010', 'BPMC002'])

In [13]:
MONITOR = {pvname:epics.PV(pvname) for pvname in PVLIST_GOOD}
sleep(1) # Wait for all to connect

In [14]:
def get_pvdata():   
    itime = isotime()
    pvdata =  {k:MONITOR[k].get() for k in MONITOR}
    return pvdata
PVDATA = get_pvdata()

# Fiter datamamps again

In [15]:
for name in BAD_NAMES:
    good_names.remove(name)

In [16]:
DM_GOOD = {}
for name, dm in DM.items():
    if name == 'tao_energy_measurements':
        # don't filter
        DM_GOOD[name] = dm
    else:
        DM_GOOD[name] = filter_datamap(dm, good_names)

# Tao conveniences

In [17]:
def tao_commands(pvdata):
    cmds = []
    for name, dm in DM_GOOD.items():
        cmds.extend(dm.as_tao(pvdata))
    return cmds

In [18]:
def save_cmds(cmds, filename='cmds.tao'): # Write to file for running with vanilla Tao
    with open(filename, 'w') as f:
        f.write('set global lattice_calc_on = F\n')
        f.write('set global plot_on = F\n')    
        for cmd in CMDS:
            f.write(cmd+'\n')
        f.write('set global lattice_calc_on = T\n')        
        f.write('set global plot_on = T\n')   

In [19]:
# Match HTR to design
def set_htr_twiss(tao):
    cmds="""
vv
vd
use dat HTR.begtwiss[1:4]
use var begtwiss[1:4]
olmdif
""".split('\n') 
    tao.cmds(cmds)
    tao.cmd('set global lattice_calc_on = T')
    tao.cmd('run')
    tao.cmd('set global plot_on = T')
#set_htr_twiss(tao)

In [20]:
# mat2 x, y for PyEmittance
def get_mats():
    mat6 = tao.matrix('Q0H01#2', 'OTR0H04')['mat6']
    mat2x = mat6[0:2, 0:2]
    mat2y = mat6[2:4, 2:4]
    return mat2x, mat2y

## Form commands using PVDATA and datamaps

In [21]:
# Master switches for element scaling
tao.cmd('set ele quad::* field_master = T')
#tao.cmd('set ele lcavity::CAVL* autoscale_amplitude = T') # field_au
if MODEL == 'sc_inj':
    tao.cmd('set ele lcavity::CAVL* autoscale_phase = T')

# Continuous loop

In [22]:
def run1():
    #sleep(.001)
    t1 = time()
    pvdata = get_pvdata()
    cmds = tao_commands(pvdata)
    tao.cmd('set global plot_on = F;set global lattice_calc_on = F')
    tao.cmds(cmds); # Apply

    tao.cmd('set global lattice_calc_on = T')
    tao.cmd('set global plot_on = T')
    #toggle_beam()
    
    dt = time()-t1
    #print(dt)    


In [23]:
%%time
run1()

CPU times: user 23.1 ms, sys: 1.71 ms, total: 24.8 ms
Wall time: 25.6 ms


In [24]:
# Set twiss
set_htr_twiss(tao)

In [25]:
%%tao
use dat orbit.x
use dat orbit.y
sc floor -5 5

-------------------------
Tao> use dat orbit.x
  orbit.x[1:171]                                 Using: 1:11 13:14 16:18 20:21 24:25
  orbit.y[1:171]                                 Using:
  orbit.charge[1:171]                            Using:
-------------------------
Tao> use dat orbit.y
  orbit.x[1:171]                                 Using: 1:11 13:14 16:18 20:21 24:25
  orbit.y[1:171]                                 Using: 1:11 13:14 16:18 20:21 24:25
  orbit.charge[1:171]                            Using:
-------------------------
Tao> sc floor -5 5
-------------------------
Tao> 


# Continuous run
    

In [ ]:
# Run forever
while True:
    run1()